In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle

In [2]:
df = pd.read_csv('reviews_dataset.tsv.zip',header=0, delimiter="\t", quoting=3)
df = df[['review','sentiment']]

In [3]:
df.shape

(25000, 2)

In [4]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [5]:
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'] = df['review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [6]:
max_features = 1000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['review'].values)
X = tokenizer.texts_to_sequences(df['review'].values)
X = pad_sequences(X)
X.shape

(25000, 1473)

In [7]:
embed_dim = 50
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(LSTM(10))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1473, 50)          50000     
                                                                 
 lstm (LSTM)                 (None, 10)                2440      
                                                                 
 dense (Dense)               (None, 2)                 22        
                                                                 
Total params: 52,462
Trainable params: 52,462
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
y = pd.get_dummies(df['sentiment']).values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 99)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(18750, 1473) (18750, 2)
(6250, 1473) (6250, 2)


In [9]:
model.fit(X_train, y_train, epochs = 5, verbose = 1)

Epoch 1/5
586/586 [==============================] - 53s 81ms/step - loss: 0.4647 - accuracy: 0.7791
Epoch 2/5
586/586 [==============================] - 50s 86ms/step - loss: 0.3468 - accuracy: 0.8552
Epoch 3/5
586/586 [==============================] - 52s 89ms/step - loss: 0.3224 - accuracy: 0.8666
Epoch 4/5
586/586 [==============================] - 50s 86ms/step - loss: 0.3196 - accuracy: 0.8634
Epoch 5/5
586/586 [==============================] - 50s 86ms/step - loss: 0.2883 - accuracy: 0.8823


In [10]:
test = ['Movie was pathetic']
test = tokenizer.texts_to_sequences(test)
test = pad_sequences(test, maxlen=X.shape[1],dtype='int32', value=0)
print(test.shape)
sentiment = model.predict(test)[0]
if(np.argmax(sentiment) == 0):
    print("Negative")
elif (np.argmax(sentiment) == 1):
    print("Positive")

(1, 1473)
Negative


In [11]:
with open('tokenizer.pickle', 'wb') as tk:
    pickle.dump(tokenizer, tk, protocol=pickle.HIGHEST_PROTOCOL)

model_json = model.to_json()
with open("model.json", "w") as js:
    js.write(model_json)

In [12]:
model.save_weights("model.h5")